v3.0 (corresponds to opentrons gibson_v1.py)
- incorporates 8-channel tip, instead of single channel in previous versions (p10 multi on left)
- incorporates p300 on the right mount to accommodate larger volumes (p300 multi on right)
- replaced reagent plate (deep well 2mL) with a reagent reservoir
- includes self-written OT2Specialization functions (open_lid, close_lid, mix, set_lid_temp, set_block_temp)

In [1]:
%reset

In [2]:
from logging import Logger
import os
import logging
import sbol3
import labop
import tyto
import uml
import json
import rdflib as rdfl
from typing import Dict, Tuple
from sbol3 import Document

from labop.execution_engine import ExecutionEngine
from labop_convert.opentrons.opentrons_specialization import OT2Specialization, REVERSE_LABWARE_MAP
from labop_convert.markdown.markdown_specialization import MarkdownSpecialization

from IPython.display import Image

/Users/jeytse/BMEY3_LabAutomationGroupProject/labOP_env/lib/python3.10/site-packages/sbol3/object.py:70: UserWarning: Using a default namespace. To set a namespace use set_namespace()
  warnings.warn('Using a default namespace.'


In [3]:
# set up the document
print('Setting up document')
doc = sbol3.Document()
sbol3.set_namespace('https://bmey3labautomation.com/')

Setting up document


In [4]:
# Import the primitive libraries
print('Importing libraries')
labop.import_library('liquid_handling')
print('... Imported liquid handling')
labop.import_library('plate_handling')
print('... Imported plate handling')
labop.import_library('spectrophotometry')
print('... Imported spectrophotometry')
labop.import_library('sample_arrays')
print('... Imported sample arrays')
labop.import_library('pcr')
print('... Imported pcr')

Importing libraries
... Imported liquid handling
... Imported plate handling
... Imported spectrophotometry
... Imported sample arrays
... Imported pcr


In [5]:
protocol = labop.Protocol('gibson_assembly_labop_v3_0')
protocol.name = "LabOP Opentrons Gibson Assembly Protocol"
doc.add(protocol)

# create the materials to be provisioned
CONT_NS = rdfl.Namespace('https://sift.net/container-ontology/container-ontology#')
OM_NS = rdfl.Namespace('http://www.ontology-of-units-of-measure.org/resource/om-2/')

PREFIX_MAP = json.dumps({"cont": CONT_NS, "om": OM_NS})

In [6]:
doc.find('https://bmey3labautomation.com/gibson_assembly_labop_v2_0')

In [7]:
# Define reagents
# do we need to define all our reagents? if yes, how do we know which URI for our reagents?
dh2o = sbol3.Component('dH2O', 'https://pubchem.ncbi.nlm.nih.gov/substance/24901740')
dh2o.name = 'Water, sterile-filtered, BioReagent, suitable for cell culture'
doc.add(dh2o)

master_mix = sbol3.Component('master_mix', 'gibsonassemblymastermix')
master_mix.name = 'Gibson Assembly Master Mix'
doc.add(master_mix)

dna_frag1 = sbol3.Component('dna_frag1', 'dnafragments1')
dna_frag1.name = 'DNA Fragment 1'
doc.add(dna_frag1)

dna_frag2 = sbol3.Component('dna_frag2', 'dnafragments2')
dna_frag2.name = 'DNA Fragment 2'
doc.add(dna_frag2)

In [8]:
# Define labware
spec_reagent_plate = labop.ContainerSpec('reagent_plate',
                                    name='Well plate for reagents',
                                    queryString=REVERSE_LABWARE_MAP['masterblock_96_wellplate_2000ul'],
                                    prefixMap=PREFIX_MAP)
spec_destination_plate = labop.ContainerSpec('destination_plate',
                                        name='Destination plate',
                                        queryString=REVERSE_LABWARE_MAP['framestar_96_aluminumblock_200ul'],
                                        prefixMap=PREFIX_MAP)
spec_tiprack_p10 = labop.ContainerSpec('tiprack_p10',
                                       name='P10 Tiprack',
                                       queryString=REVERSE_LABWARE_MAP['opentrons_96_tiprack_10ul'],
                                        prefixMap=PREFIX_MAP)
spec_tiprack_p300 = labop.ContainerSpec('tiprack_p300',
                                        name='P300 Tiprack',
                                        queryString=REVERSE_LABWARE_MAP['opentrons_96_tiprack_300ul'],
                                        prefixMap=PREFIX_MAP)
p10_multi = sbol3.Agent('p10_multi',
                        name='P10 Multi')
p300_multi = sbol3.Agent('p300_multi',
                         name='P300 Multi')
thermocycler = sbol3.Agent('thermocycler_module',
                           name = 'Thermocycler Module')
doc.add(spec_tiprack_p10)
doc.add(spec_tiprack_p300)
doc.add(spec_reagent_plate)
doc.add(spec_destination_plate)
doc.add(p10_multi)
doc.add(p300_multi)
doc.add(thermocycler)

tiprack_p10 = protocol.primitive_step('EmptyContainer',
                                      specification=spec_tiprack_p10)
tiprack_p300 = protocol.primitive_step('EmptyContainer',
                                       specification=spec_tiprack_p300)
reagent_plate = protocol.primitive_step('EmptyContainer',
                                        specification=spec_reagent_plate)
destination_plate = protocol.primitive_step('EmptyContainer',
                                            specification=spec_destination_plate)

In [9]:
# Load OT2 robot with labware
left_pipette = protocol.primitive_step('ConfigureRobot', 
                                      instrument=p10_multi,
                                      mount='left')
right_pipette = protocol.primitive_step('ConfigureRobot',
                                        instrument=p300_multi,
                                        mount='right')
thermocycler_module = protocol.primitive_step('ConfigureRobot',
                                              instrument=thermocycler,
                                              mount='7')
load_tiprack_p10 = protocol.primitive_step('LoadRackOnInstrument',
                                           rack=spec_tiprack_p10,
                                           coordinates='1')
load_tiprack_p300 = protocol.primitive_step('LoadRackOnInstrument',
                                            rack=spec_tiprack_p300,
                                            coordinates='2')
load_reagent_plate = protocol.primitive_step('LoadRackOnInstrument',
                                             rack=spec_reagent_plate,
                                             coordinates='4')
load_destination_plate = protocol.primitive_step('LoadContainerOnInstrument',
                                                 specification=spec_destination_plate,
                                                 instrument=thermocycler,
                                                 slots='A1:H12') # 96 wells

In [10]:
# Set up target samples
assembly_reaction = protocol.primitive_step('PlateCoordinates',
                                            source=destination_plate.output_pin('samples'),
                                            coordinates='A1') # A1:H1 for multi-channel
negative_control = protocol.primitive_step('PlateCoordinates',
                                           source=destination_plate.output_pin('samples'),
                                           coordinates='A2') # A2:H2 for multi-channel

In [11]:
# Set up reagents - assuming reagents are loaded onto reagent plate manually
master_mix_well = protocol.primitive_step('PlateCoordinates',
                                          source=reagent_plate.output_pin('samples'),
                                          coordinates='A1') # A1:H1 for multi-channel
dh2o_well = protocol.primitive_step('PlateCoordinates',
                                    source=reagent_plate.output_pin('samples'),
                                    coordinates='A2') # A2:H2 for multi-channel
dna_frag1_well = protocol.primitive_step('PlateCoordinates',
                                         source=reagent_plate.output_pin('samples'),
                                         coordinates='A3') # A3:H3 for multi-channel
dna_frag2_well = protocol.primitive_step('PlateCoordinates',
                                         source=reagent_plate.output_pin('samples'),
                                         coordinates='A4') # A4:H4 for multi-channel

In [12]:
# Protocol

hold = protocol.primitive_step('Hold',
                               temperature=sbol3.Measure(4,tyto.OM.degree_Celsius))
hold.name = '1. Hold at 4 (ice)'

transfer = protocol.primitive_step('Transfer',
                                   source=dh2o_well.output_pin('samples'),
                                   destination=assembly_reaction.output_pin('samples'),
                                   pipette='right', # corresponds to right mount (p300_multi)
                                   amount=sbol3.Measure(16,tyto.OM.microliter))
transfer.name = '2. Add 16 ul deionized H2O for assembly reaction'

transfer = protocol.primitive_step('Transfer',
                                   source=dh2o_well.output_pin('samples'),
                                   destination=negative_control.output_pin('samples'),
                                   pipette='right', 
                                   amount=sbol3.Measure(36,tyto.OM.microliter))
transfer.name = '3. Add 36 ul deionized H2O for negative control'

transfer = protocol.primitive_step('Transfer',
                                   source=master_mix_well.output_pin('samples'),
                                   destination=assembly_reaction.output_pin('samples'),
                                   pipette='right',
                                   amount=sbol3.Measure(20,tyto.OM.microliter))
transfer.name = '4. Add 20 ul Gibson Assembly Master Mix for assembly reaction'

transfer = protocol.primitive_step('Transfer',
                                   source=dna_frag1_well.output_pin('samples'),
                                   destination=assembly_reaction.output_pin('samples'),
                                   pipette='left',
                                   amount=sbol3.Measure(2,tyto.OM.microliter))
transfer.name = '5. Add 2 ul DNA fragment 1 for assembly reaction'

transfer = protocol.primitive_step('Transfer',
                                   source=dna_frag1_well.output_pin('samples'),
                                   destination=negative_control.output_pin('samples'),
                                   pipette='left',
                                   amount=sbol3.Measure(2,tyto.OM.microliter))
transfer.name = '6. Add 2 ul DNA fragment 1 for negative control'

transfer = protocol.primitive_step('Transfer',
                                   source=dna_frag2_well.output_pin('samples'),
                                   destination=assembly_reaction.output_pin('samples'),
                                   pipette='left',
                                   amount=sbol3.Measure(2,tyto.OM.microliter))
transfer.name = '7. Add 2 ul DNA fragment 2 for assembly reaction'

transfer = protocol.primitive_step('Transfer',
                                   source=dna_frag2_well.output_pin('samples'),
                                   destination=negative_control.output_pin('samples'),
                                   pipette='left',
                                   amount=sbol3.Measure(2,tyto.OM.microliter))
transfer.name = '8. Add 2 ul DNA fragment 2 for negative control'

mix = protocol.primitive_step('PipetteMix',
                              samples=assembly_reaction.output_pin('samples'),
                              amount=sbol3.Measure(20,tyto.OM.microliter),
                              cycleCount='5',
                              pipette='right')
mix.name = '9. Mix assembly reaction (volume 20 ul, 5 repetitions)'

mix = protocol.primitive_step('PipetteMix',
                              samples=negative_control.output_pin('samples'),
                              amount=sbol3.Measure(20,tyto.OM.microliter),
                              cycleCount='5',
                              pipette='right')
mix.name = '10. Mix negative control (volume 20 ul, 5 repetitions)'

incubate = protocol.primitive_step('Incubate',
                                   duration=sbol3.Measure(15,tyto.OM.minute),
                                   temperature=sbol3.Measure(50,tyto.OM.degree_Celsius))
incubate.name = '11. Incubate at 50 C for 15 minutes'

In [13]:
filename = 'ot2_gibson_assembly_labop_v3.0'
# filename = os.path.join(out_dir, "ot2_gibson_assembly_labop") # out_dir currently points to test
agent = sbol3.Agent('ot2_machine', name='OT2 machine')
ee = ExecutionEngine(specializations=[OT2Specialization(filename)],failsafe=False)
parameter_values = []
execution = ee.execute(protocol, agent, id='test_execution')

https://bmey3labautomation.com/test_execution/CallBehaviorExecution19 https://bmey3labautomation.com/gibson_assembly_labop_v3_0/CallBehaviorAction19 param = source: http://bioprotocols.org/labop#SampleCollection 
https://bmey3labautomation.com/test_execution/CallBehaviorExecution19 https://bmey3labautomation.com/gibson_assembly_labop_v3_0/CallBehaviorAction19 param = source: http://bioprotocols.org/labop#SampleCollection 
https://bmey3labautomation.com/test_execution/ActivityEdgeFlow36 src = https://bmey3labautomation.com/gibson_assembly_labop_v3_0/CallBehaviorAction19/InputPin2 param = source: http://bioprotocols.org/labop#SampleCollection 
https://bmey3labautomation.com/test_execution/ActivityEdgeFlow58 src = https://bmey3labautomation.com/gibson_assembly_labop_v3_0/CallBehaviorAction19/InputPin1 param = source: http://bioprotocols.org/labop#SampleCollection 
https://bmey3labautomation.com/test_execution/ActivityNodeExecution23 https://bmey3labautomation.com/gibson_assembly_labop_v3_